In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
data = [
    # Positive Entries
    ("Этот фильм был потрясающим! Отличная игра актеров и захватывающий сюжет.", "positive"),
    ("Этот фильм был великолепен! Великолепная актёрская игра и увлекательный сюжет.", "positive"),
    ("Фильм оказался отличным! Потрясающие актёры и интересный сюжет.", "positive"),
    ("Какой потрясающий фильм! Замечательные актёры и увлекательный сюжет.", "positive"),
    ("Фильм превзошел все ожидания! Невероятно захватывающий.", "positive"),
    ("Прекрасный фильм, полный удивительных моментов и отличной игры актеров.", "positive"),
    ("Фильм вызвал только положительные эмоции, актеры сыграли на высоте.", "positive"),
    ("Фильм был исключительно хорош, с отличной режиссурой и актерской игрой.", "positive"),
    ("Великолепная кинолента, полная драматизма и напряженных моментов.", "positive"),
    ("Фильм оставил глубокое впечатление, благодаря своему сюжету и исполнению.", "positive"),
    ("Отличное кино, стоит посмотреть каждому!", "positive"),
    ("Фильм поразил своей глубиной и актерским мастерством.", "positive"),
    ("Это кинопроизведение является истинным шедевром!", "positive"),
    ("Фильм невероятно увлекателен, с множеством неожиданных поворотов.", "positive"),
    ("Кино, которое действительно стоит вашего времени.", "positive"),
    ("Незабываемый фильм с захватывающим сюжетом и великолепной игрой актеров.", "positive"),
    ("Кинолента, оставляющая после себя только позитивные эмоции.", "positive"),
    ("Этот фильм заслуживает высших похвал и множество наград!", "positive"),
    ("Впечатляющий фильм, который вызывает массу положительных чувств.", "positive"),
    ("Смотреть этот фильм - одно удовольствие, рекомендую всем!", "positive"),
    ("Фильм, который перевернул мое представление о кино.", "positive"),
    ("Картина сильно впечатлила, несомненно, одно из лучших произведений года.", "positive"),
    ("Прекрасное кино, полное эмоций и незабываемых сцен.", "positive"),
    ("Фильм, который я буду рекомендовать всем своим друзьям.", "positive"),
    ("Кинопроизведение, которое затрагивает душу и заставляет задуматься.", "positive"),
    ("Фильм с необыкновенной атмосферой и глубоким смыслом.", "positive"),
    ("Это кино не просто хорошо, оно великолепно в каждом аспекте!", "positive"),
    ("Увлекательный фильм, который не дает заскучать ни на минуту.", "positive"),
    ("Замечательная кинокартина с отличным сюжетом и исполнением.", "positive"),
    ("Фильм, который доставляет массу удовольствий и положительных впечатлений.", "positive"),
    ("Кинолента, заслуживающая высоких оценок и обсуждения.", "positive"),
    ("Превосходное кино, которое оставляет после себя только положительные впечатления.", "positive"),
    ("Фильм, который вызывает желание смотреть его снова и снова.", "positive"),

    # Negative Entries
    ("Ужасный сервис в этом ресторане. Никогда больше сюда не вернусь.", "negative"),
    ("Ужасное обслуживание в этом ресторане. Больше не приду сюда.", "negative"),
    ("Плохой сервис в ресторане. Не рекомендую туда ходить.", "negative"),
    ("Сервис был ужасен. Не стану возвращаться в этот ресторан.", "negative"),
    ("Ужасное обслуживание, никогда не заказывайте здесь.", "negative"),
    ("Посещение этого ресторана оставило самые негативные впечатления.", "negative"),
    ("Неприятный опыт в этом заведении. Не планирую возвращаться.", "negative"),
    ("Ресторан оставил желать лучшего, сервис был ниже среднего.", "negative"),
    ("Плохое обслуживание и качество еды оставили неприятный осадок.", "negative"),
    ("Недовольство сервисом в этом месте. Буду избегать в будущем.", "negative"),
    ("Ужасный опыт обслуживания, который полностью испортил вечер.", "negative"),
    ("Сервис был медленным и невнимательным, не рекомендую.", "negative"),
    ("Обслуживание было настолько плохим, что это испортило все впечатление.", "negative"),
    ("Неприятный персонал и ужасное обслуживание в этом ресторане.", "negative"),
    ("Ужасное качество обслуживания, не соответствует ожиданиям.", "negative"),
    ("Недовольны сервисом, еда была подана холодной.", "negative"),
    ("Неприемлемый уровень сервиса, не советую сюда ходить.", "negative"),
    ("Отвратительное обслуживание, не вернусь сюда больше.", "negative"),
    ("Очень плохой сервис, долго ждали заказ.", "negative"),
    ("Сервис оставляет желать лучшего, не рекомендую.", "negative"),
    ("Плохой опыт в этом ресторане, сервис был ниже всякой критики.", "negative"),
    ("Разочарован качеством обслуживания в этом заведении.", "negative"),
    ("Плохое обслуживание, неприятный персонал, не рекомендую.", "negative"),
    ("Самый худший опыт обслуживания за последнее время.", "negative"),
    ("Ужасный вечер из-за плохого сервиса и невкусной еды.", "negative"),
    ("Сервис был настолько плох, что мы решили уйти.", "negative"),
    ("Отсутствие профессионализма в обслуживании, не советую сюда ходить.", "negative"),
    ("Плохое обслуживание, персонал не обращал на нас внимание.", "negative"),
    ("Ужасный опыт, сервис был медленным и невежливым.", "negative"),
    ("Неприятное обслуживание, долгое ожидание и некачественная еда.", "negative"),
    ("Сервис был ужасен, персонал был грубым и не внимательным.", "negative"),
    ("Ресторан с ужасным сервисом и отсутствием вкусной еды.", "negative"),
    ("Плохое обслуживание, ждать еду пришлось очень долго.", "negative"),

    # Neutral Entries
    ("Книга была довольно интересной, но слишком затянутой.", "neutral"),
    ("Книга интересная, но длинновата.", "neutral"),
    ("Книга захватывает, но чересчур долга.", "neutral"),
    ("Интересное чтение, однако затянутое.", "neutral"),
    ("Книга имеет свои моменты, но в целом растянута.", "neutral"),
    ("Чтение было захватывающим, но нужно было сократить.", "neutral"),
    ("Книга была увлекательной, но могла быть кратче.", "neutral"),
    ("Текст интересен, но излишне подробен.", "neutral"),
    ("Чтение показалось занимательным, но перегруженным деталями.", "neutral"),
    ("Книга привлекла внимание, но местами скучновата.", "neutral"),
    ("Произведение заинтриговало, однако оказалось слишком многословным.", "neutral"),
    ("Книга держит в напряжении, но кажется перенасыщенной.", "neutral"),
    ("История местами увлекательная, но затянута.", "neutral"),
    ("Чтение было приятным, хотя книга и могла бы быть короче.", "neutral"),
    ("Текст читается легко, но имеет лишние разделы.", "neutral"),
    ("Произведение интересное, но имеет лишнюю длину.", "neutral"),
    ("Книга местами увлекательна, но часто теряет динамику.", "neutral"),
    ("Интересный сюжет, но излишне растянут.", "neutral"),
    ("Роман привлекает внимание, но мог быть написан более лаконично.", "neutral"),
    ("Прочтение было занимательным, но книга длиннее, чем необходимо.", "neutral"),
    ("Текст неплохой, но содержит много лишних подробностей.", "neutral"),
    ("Книга держит внимание, однако могла бы быть сжатее.", "neutral"),
    ("История затягивает, но книга чрезмерно длинная.", "neutral"),
    ("Книга интересна, но содержит много лишнего.", "neutral"),
    ("Произведение вызывает интерес, но его можно было бы укоротить.", "neutral"),
    ("Книга читается с интересом, но местами скучна.", "neutral"),
    ("Сюжет занимателен, но книга нуждается в редактировании.", "neutral"),
    ("Текст привлекает, но местами перегружен.", "neutral"),
    ("Чтение увлекает, но книга кажется переусложненной.", "neutral"),
    ("Интересная книга, однако некоторые части излишни.", "neutral"),
    ("Прочтение было неплохим, но книга чуть перегружена.", "neutral"),
    ("Книга имела хорошие моменты, но слишком долгая.", "neutral"),
    ("История была интересна, но развернута слишком широко.", "neutral")
]

In [9]:
russian_stopwords = stopwords.words("russian")
stemmer = SnowballStemmer("russian")

In [11]:
def preprocess(text):
    tokens = [stemmer.stem(word.lower()) for word in nltk.word_tokenize(text) if word.lower() not in russian_stopwords]
    return " ".join(tokens)

In [13]:
vectorizer = TfidfVectorizer(preprocessor=preprocess)
X = vectorizer.fit_transform([text for text, _ in data])
y = [sentiment for _, sentiment in data]

model = MultinomialNB()
model.fit(X, y)

MultinomialNB()

In [20]:
test_text = "История местами увлекательная, но затянута."
processed_text = preprocess(test_text)
vector = vectorizer.transform([processed_text])
sentiment = model.predict(vector)[0]
print(f"Sentiment for '{test_text}' is '{sentiment}'")

Sentiment for 'История местами увлекательная, но затянута.' is 'neutral'
